In [23]:
import pandas as pd

xls = pd.read_excel("numericalBetterData.xlsx", sheet_name="Sheet1")
y = xls.iloc[:, 1:4]  #column 0 is just numerical indices for some reason
X = xls.iloc[:, 4:]
X=X.to_numpy()
y=y.to_numpy()
from sklearn.preprocessing import StandardScaler
#Omitted train_test_split
scaler = StandardScaler()
X = scaler.fit_transform(X) # Why omit test data when we are doing predictions on new data?

from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2, weights='uniform') 
neigh.fit(X, y)


KNeighborsRegressor(n_neighbors=2)

In [24]:
X

array([[-1.46629495, -0.451523  ,  0.12590584, ..., -0.30946358,
        -1.4538384 ,  0.82138994],
       [ 1.78304767, -1.30611843, -1.90251522, ...,  0.92282801,
         0.88194675,  0.82138994],
       [ 1.69818884, -0.84141272, -2.09693626, ...,  0.23372667,
        -1.67744746, -1.21744855],
       ...,
       [-1.61755125,  0.32853694, -0.19373928, ..., -1.26341147,
         0.31749955,  0.82138994],
       [-1.81854266,  0.22385576, -0.14722251, ...,  0.25351699,
        -0.9829436 ,  0.82138994],
       [-1.05890595, -0.00614178, -0.70031138, ..., -1.11509217,
         1.31092662,  0.82138994]])

In [27]:
import cv2
import numpy as np


#This should be a path to a camera or a video file.
cap = cv2.VideoCapture('test2.mp4') # 0 is my phone camera. 

def flatten(xss):
    return [x for xs in xss for x in xs]

while (cap.isOpened()):
    ret0, frame = cap.read()

#split the images by channel        
    imRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imHSV=cv2.cvtColor(imRGB, cv2.COLOR_RGB2HSV)
    h,s,v=cv2.split(imHSV)
    r,g,b=cv2.split(imRGB)
    
#create filter for the blue stars - this one is easy    
    bMin=(0,0,120)
    bMax=(255,100,255)
    blueMask=cv2.inRange(imRGB, bMin, bMax)
    
#The green ones were problematic as they cast a lot of light    
    gMin=(0,100,0)
    gMax=(255,255,100)
    greenMask=cv2.inRange(imRGB, gMin, gMax)

#I had to use a value threshold as well to get rid of the reflected light    
    vThresh=200
    ret1,thresh = cv2.threshold(v,vThresh,255,cv2.THRESH_BINARY)
    
#Combine the masks with bitwise_and    
    greenValMask=cv2.bitwise_and(thresh, greenMask)

#Look at the other notebook to notice how the camera oversaturation forms a nice circle at the centre of the LED.
# Use a Hough circle transform to find the centre of that circle.

    data=[]
    imCentre=[imRGB.shape[1]/2.,imRGB.shape[0]/2.]
    ims = [greenValMask, blueMask]
    colour=[1, 0]
    xC,yC=imCentre                  
    for i in range(0,2):
        circles = cv2.HoughCircles(ims[i],cv2.HOUGH_GRADIENT,1.5,200,param1=100,param2=10,minRadius=1,maxRadius=20)
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
#Find the distance and displacement of each circle's centre from the centre of the image.  
            diffX=x-xC                         
            diffY=y-yC
            diffR=np.sqrt(diffX**2+diffY**2)   #convert to distance
            diffR=np.round(diffR,decimals=3)
            
#Add it to the list, along with the colour of the LED.            
            data.append([diffR, diffX, diffY, colour[i]])
    sortedList=sorted(data)
    top3=sortedList[:3]     
    flat3=np.array(flatten(top3)).reshape(1,-1)
    #df=pd.DataFrame()
    #df.columns=X.columns.values
    flat3.reshape(-1,1)
    xT=scaler.transform(flat3)
    yPred=neigh.predict(xT)
    print(yPred)

[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[  0. 450.  80.]]
[[350.   0. 150.]]
[[350.   0. 150.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[350.   0. 150.]]
[[350.   0. 150.]]
[[375.   0. 100.]]
[[375.   0. 100.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[300. 200. 165.]]
[[400.   0. 260.]]
[[400.   0. 260.]]
[[400.   0. 260.]]
[[400.   0. 260.]]
[[400.   0. 

error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
print(flat3)